# Эксперимент 1 - исследование датасета

Используем датасет [с гитлаба](https://github.com/dkulagin/kartaslov/blob/master/dataset/orfo_and_typos/orfo_and_typos.L1_5%2BPHON.csv)

In [1]:
import pandas as pd

In [2]:
url = "https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/orfo_and_typos/orfo_and_typos.L1_5%2BPHON.csv"

In [3]:
df = pd.read_csv(url)

In [4]:
df

,CORRECT;MISTAKE;WEIGHT
0,болота;болотро;0.5
1,болота;балото;0.2652
2,болота;боллото;0.0909
3,болота;болотоэ;0.0909
4,болото;палатаа;0.5
...,...
90967,догадливый;догадливвый;0.5
90968,пуховый;пуховык;0.5
90969,ослица;ослиа;0.25
90970,ослик;ослиа;0.5


### Вывод

Нет предложений, только единичные слова

Ищем датасет дальше

Интересная статья с рецептами:

https://habr.com/ru/companies/sberdevices/articles/763932/

In [5]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset


dataset: DatasetDict = load_dataset("ai-forever/spellcheck_benchmark", "RUSpellRU")

c:\lang_check\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset["train"]

Dataset({
    features: ['source', 'correction', 'domain'],
    num_rows: 2000
})

SAGE бенчмаркинг

https://habr.com/ru/companies/sberdevices/articles/763932/

import os
import torch
from sage.utils import DatasetsAvailable
from sage.spelling_correction import AvailableCorrectors
from sage.spelling_correction import T5ModelForSpellingCorruption

corrector_fred_95m = T5ModelForSpellingCorruption.from_pretrained(
    AvailableCorrectors.sage_fredt5_distilled_95m.value
)
corrector_mt5 = T5ModelForSpellingCorruption.from_pretrained(
    AvailableCorrectors.sage_mt5_large.value
)

corrector_fred_95m.model.to(torch.device("cuda:0"))
corrector_mt5.model.to(torch.device("cuda:0"))

metrics = corrector_fred_95m.evaluate(
    "RUSpellRU", metrics=["errant", "ruspelleval"], batch_size=32
)
print(metrics)

metrics = corrector_mt5.evaluate(
    "/content/sage/data/example_data/jfleg", metrics=["ruspelleval"], batch_size=16
)
print(metrics)

**Важно**

При работе на windows может быть ошибка с отсутствием `SIGALRM`

В этом случае придётся залезть в исходники и убрать все упоминания `SIGALRM`

In [7]:
from sage.evaluation.scorer import Scorer

s = Scorer(load_errant=False)


s.score(
    ["спел Кейс ее .", "спел Кей ее ."],
    ["спелл кейс её !", "спелл кейс её !"],
    ["спел кейс её .", "спелл Кейс ее !"],
    metrics=["ruspelleval"],
)

Calculating words metric: 100%|██████████| 2/2 [00:00<00:00, 153.93it/s]


{'Precision': 100.0, 'Recall': 66.67, 'F1': 80.0}